In [1]:
from typing import List, Optional, Generator
import pandas as pd
import numpy as np
import sqlite3
import re
import io
import math
import collections
import itertools
import functools
import random
import string
import tqdm
import bisect
import heapq

conn = sqlite3.connect(":memory:")

def regexp(expr, item):
    reg = re.compile(expr)
    return reg.search(item) is not None

def read_lc_df(s: str, dtypes: dict[str, str]=dict()) -> pd.DataFrame:
    temp = pd.read_csv(io.StringIO(s), sep="|", skiprows=2)
    temp = temp.iloc[1:-1, 1:-1]
    temp.columns = temp.columns.map(str.strip)
    temp = temp.map(lambda x: x if type(x) != str else None if x.strip() == 'null' else x.strip())
    temp = temp.astype(dtypes)
    return temp

conn.create_function("REGEXP", 2, regexp)

#### Helper for Binary tree problems

In [2]:
class BinaryTreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

    def to_list(self):
        to_visit = [self]
        visited = []
        while len(to_visit) > 0:
            curr = to_visit.pop(0)
            if curr:
                to_visit.append(curr.left)
                to_visit.append(curr.right)
                visited.append(curr.val)
            else:
                visited.append(curr)

        while visited and not visited[-1]:
            visited.pop()

        return visited

    def __str__(self):
        return str(self.val)

    @staticmethod
    def from_array(nums: list[int|None]):
        '''Create a Tree from a list of nums. Returns the root node.'''
        if len(nums) == 0:
            return None
        elif len(nums) == 1:
            return BinaryTreeNode(nums[0])
        else:
            forest = [BinaryTreeNode(nums[0])]
            parent_idx = -1
            for i in range(1, len(nums)):

                curr = None
                if nums[i] is not None:
                    curr = BinaryTreeNode(nums[i])
                    forest.append(curr)

                if i % 2 == 1:
                    parent_idx += 1
                    forest[parent_idx].left = curr
                else:
                    forest[parent_idx].right = curr

        return forest[0]

#### Helper for Singly Linked lists

In [3]:
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

    def __str__(self):
        return str(self.val)

    @staticmethod
    def to_singly_linked_list(nums: list[int]):
        root = prev = None
        for n in nums:
            curr = ListNode(n)
            # Init once
            if not root:
                root = curr
            if prev:
                prev.next = curr
            prev = curr

        return root

    def to_list(self) -> list[int]:
        result = []
        curr = self
        while curr:
            result.append(curr.val)
            curr = curr.next
        return result

Codeforces practice

In [4]:
# https://codeforces.com/problemset/problem/1984/B
def large_addition(N: int) -> bool:
    if N < 10:
        return False
    else:
        N_str: str = str(N)
        length = len(N_str)
        for i in range(length):
            if i == 0 and N_str[i] != '1':
                return False
            elif i == length - 1 and N_str[i] == '9':
                return False
            elif i not in (0, length - 1) and N_str[i] == '0':
                return False

        return True

# Testing the solution
assert large_addition(1393938) == True
assert large_addition(200) == False
assert large_addition(10) == True

LC Virtual: 25th June 2024
https://leetcode.com/contest/smarking-algorithm-contest-3/

In [5]:
# Q1
# https://www.youtube.com/watch?v=x8T11yRLIyY
def minMoves(nums: list[int]) -> int:
    """
    Intution: Instead of increasing all n - 1 elements by 1 each time, try to decrement
    the max element to the smallest element. Relative distances between numbers are the same.
    """
    nums.sort()
    result = 0
    for i in range(len(nums) - 1, -1, -1):
        result += nums[i] - nums[0]
    return result

# Testing the solution
assert minMoves([1,2,3,4]) == 6
assert minMoves([1,2,3]) == 3

In [6]:
# Q2
def numberOfBoomerangs(points: list[list[int]]) -> int:
    def distance(pt1: list[int], pt2: list[int]) -> float:
        return math.sqrt(((pt1[0] - pt2[0]) ** 2) + ((pt1[1] - pt2[1]) ** 2))

    boommerangs: int = 0
    N = len(points)
    for i in range(N):
        curr = points[i]
        distances: dict[float, int] = dict()
        for j in range(N):
            if j != i:
                dist = distance(points[i], points[j])
                distances[dist] = distances.get(dist, 0) + 1

        for count in distances.values():
            boommerangs += math.perm(count, 2)

    return boommerangs

# Testing the solution
assert numberOfBoomerangs([[0,0],[1,0],[2,0]]) == 2
assert numberOfBoomerangs([[1,1],[2,2],[3,3]]) == 2
assert numberOfBoomerangs([[1,1]]) == 0

In [7]:
# Q3
def findMinArrowShots(points: list[list[int]]) -> int:
    # Sort by end and merge the start, end
    points.sort(key=lambda x: (x[1], x[0]))

    # Merge the intervals
    merged_intervals: list[tuple[int, int]] = []
    for start, end in points:
        if merged_intervals and merged_intervals[-1][1] >= start:
            merged_intervals[-1] = max(merged_intervals[-1][0], start), min(merged_intervals[-1][1], end)
        else:
            merged_intervals.append((start, end))

    return len(merged_intervals)

# Testing the solution
assert findMinArrowShots([[10,16],[2,8],[1,6],[7,12]]) == 2
assert findMinArrowShots([[1,2],[3,4],[5,6],[7,8]]) == 4
assert findMinArrowShots([[1,2],[2,3],[3,4],[4,5]]) == 2

In [8]:
# Q4
def numberOfArithmeticSlicesBrute(nums: list[int]) -> int:
    def is_valid(sub: list[int]) -> bool:
        if len(sub) < 3:
            return False
        else:
            diff = sub[1] - sub[0]
            for i in range(len(sub) - 1):
                if sub[i + 1] - sub[i] != diff:
                    return False
            return True

    count = 0
    def backtrack(i: int, subseq: list[int]):
        if i == N:
            count += is_valid(subseq)
        else:
            subseq.append(nums[i])
            backtrack(i + 1, subseq)
            subseq.pop()
            backtrack(i + 1, subseq)

    N = len(nums)
    backtrack(0, [])
    return count

In [9]:
# Upsolving
# https://www.youtube.com/watch?v=YIMwwT9JdIE
def numberOfArithmeticSlices(nums: list[int]) -> int:
    """
    Ending at index 'j' what is maximum number of arthimetic subsequences that I can form >= 2
    """
    N = len(nums)
    dp: list[collections.defaultdict[int, int]] = [collections.defaultdict(int) for _ in range(N)]
    sub_count = dp_sum = 0
    for j in range(N):
        for i in range(j):
            delta = nums[i] - nums[j]
            dp[j][delta] += dp[i][delta] + 1
            sub_count, dp_sum = sub_count + 1, dp_sum + dp[i][delta] + 1

    return dp_sum - sub_count

# Testing the solution
assert numberOfArithmeticSlices([2,4,6,8,10]) == 7
assert numberOfArithmeticSlices([7,7,7,7,7]) == 16

LC VC: June 26th 2024

In [10]:
# Q4: Pattern 132
# https://www.youtube.com/watch?v=q5ANAl8Z458
# https://leetcode.com/problems/132-pattern/submissions/1301126531/
def find132pattern(nums: list[int]) -> bool:
    N = len(nums)

    # Monotonic stack, storing min values till index - value, min pairs
    stack: list[tuple[int, int]] = []
    min_: int = nums[0]

    for n in nums:
        while stack and stack[-1][0] <= n:
            stack.pop()

        if stack and stack[-1][1] < n:
            return True

        stack.append((n, min_))
        min_ = min(min_, n)

    return False

# Testing the solution
assert find132pattern([1,2,3,4]) == False
assert find132pattern([3,1,4,2]) == True
assert find132pattern([-1,3,2,0]) == True
assert find132pattern([3,1,2]) == False

Balance a Binary Search tree

In [11]:
# https://leetcode.com/problems/balance-a-binary-search-tree/submissions/1301150197
def balanceBST(root: BinaryTreeNode) -> BinaryTreeNode|None:
    def createBalancedBST(nodes_: list[int]) -> BinaryTreeNode|None:
        N = len(nodes_)
        if N == 0:
            return None
        else:
            mid = N // 2
            root_: BinaryTreeNode = BinaryTreeNode(nodes_[mid])
            root_.left = createBalancedBST(nodes_[:mid])
            root_.right = createBalancedBST(nodes_[mid + 1:])
            return root_

    # 1. Retrieve all elements in-order traversal (left, root, right), this returns an array in sorted order
    curr: BinaryTreeNode = root
    nodes: list[int] = []
    stack: list[BinaryTreeNode] = []
    while True:
        while curr:
            stack.append(curr)
            curr = curr.left
        if stack:
            curr = stack.pop()
            nodes.append(curr.val)
            curr = curr.right
        else:
            break

    # 2. Create a BST from the nodes: Take the middle element as the root, create left and right subtree recursively
    return createBalancedBST(nodes)

# Testing the solution
print(balanceBST(BinaryTreeNode.from_array([1,None,2,None,3,None,4,None,None])).to_list())

[3, 2, 4, 1]


LC Practice

In [12]:
def subarraySum(nums: list[int], k: int) -> int:
    prefix: collections.defaultdict = collections.defaultdict(int)
    prefix[0], count, psum = 1, 0, 0
    for n in nums:
        psum += n
        count += prefix[psum - k]
        prefix[psum] += 1

    return count

# Testing the solution
assert subarraySum([1,2,3], 3) == 2
assert subarraySum([1,1,1], 2) == 2

In [13]:
# https://leetcode.com/problems/continuous-subarray-sum/submissions/1301461461/
def checkSubarraySum(nums: list[int], k: int) -> bool:
    N = len(nums)
    prefix: dict[int, int] = {0: -1}
    psum = 0
    for i in range(N):
        psum = (psum + nums[i]) % k
        if i - prefix.get(psum, i) >= 2:
            return True
        prefix[psum] = min(prefix.get(psum, i), i)

    return False

# Testing the solution
assert checkSubarraySum([23,2,4,6,7], 6) == True
assert checkSubarraySum([0], 1) == False
assert checkSubarraySum([23,2,6,4,7,13], 13) == False
assert checkSubarraySum([23,2,6,4,7,13,13], 13) == True
assert checkSubarraySum([1,0], 2) == False
assert checkSubarraySum([1,2,3,4], 6) == True

Check if array is sorted and rotated

In [14]:
def checkArraySortedAndRotated(nums: list[int]) -> bool:
    N = len(nums)
    count = 0
    for i in range(N - 1):
        count += nums[i + 1] < nums[i]
    return count == 0 or (count == 1 and nums[0] > nums[-1])

# Testing the solution
assert checkArraySortedAndRotated([3,4,5,1,2]) == True
assert checkArraySortedAndRotated([2,1,3,4]) == False
assert checkArraySortedAndRotated([3,2,1]) == False

All ancestors of a node in a DAG

In [15]:
# https://leetcode.com/problems/all-ancestors-of-a-node-in-a-directed-acyclic-graph/submissions/1304043339
def getAncestors(N: int, edges: list[list[int]]) -> list[list[int]]:
    @functools.cache
    def DFS(n: int) -> set[int]:
        ancestors: set[int] = set()
        for next_ in adjl[n]:
            ancestors.add(next_)
            for next_ancestor in DFS(next_):
                ancestors.add(next_ancestor)
        return ancestors

    adjl: collections.defaultdict[int, list[int]] = collections.defaultdict(list)
    for n1, n2 in edges:
        adjl[n2].append(n1)

    result: list[list[int]] = [[] for _ in range(N)]
    for i in range(N):
        result[i] = sorted(DFS(i))

    return result

# Testing the solution
assert getAncestors(8, [[0,3],[0,4],[1,3],[2,4],[2,7],[3,5],[3,6],[3,7],[4,6]]) == [[],[],[],[0,1],[0,2],[0,1,3],[0,1,2,3,4],[0,1,2,3]]
assert getAncestors(5, [[0,1],[0,2],[0,3],[0,4],[1,2],[1,3],[1,4],[2,3],[2,4],[3,4]]) == [[],[0],[0,1],[0,1,2],[0,1,2,3]]

LC Contest: 30th June 2024

In [16]:
# Q1
def maxHeightOfTriangle(red: int, blue: int) -> int:
    @functools.cache
    def backtrack(i: int) -> tuple[int, int]:
        if i == 0:
            return 0, 0
        else:
            prev = backtrack(i - 1)
            if i % 2 == 0:
                return prev[0], prev[1] + i
            else:
                return prev[0] + i, prev[1]

    red, blue = sorted([red, blue])
    low, high = 1, 20
    while low <= high:
        mid = (low + high) // 2
        r, b = sorted(backtrack(mid))
        if r <= red and b <= blue:
            low = mid + 1
        else:
            high = mid - 1

    return high

# Testing the solution
assert maxHeightOfTriangle(10, 1) == 2
assert maxHeightOfTriangle(1, 1) == 1
assert maxHeightOfTriangle(2, 1) == 2
assert maxHeightOfTriangle(2, 4) == 3

In [17]:
# Q2
def maximumLength(nums: list[int]) -> int:
    """
    Either all consequetive element pairs should sum up to an even number or to an odd number

    If all elements are to sum to an even number: either all should be even or all numbers should be odd => max(odd_count, even_count)
    If all elements are to sum to an odd number: greedily check validity (consequetive elements should be odd-even or even-odd)
    """
    odd_sum_length = odd_count = even_count = 0
    prev = 0 if nums[0] % 2 else 1
    for i in range(len(nums)):
        if nums[i] % 2 == 0:
            even_count += 1
        else:
            odd_count += 1

        if nums[i] % 2 != prev:
            odd_sum_length, prev = odd_sum_length + 1, nums[i] % 2

    return max(odd_sum_length, odd_count, even_count)

# Testing the solution
assert maximumLength([1,2,3,4]) == 4
assert maximumLength([1,2,1,1,2,1,2]) == 6
assert maximumLength([1,3]) == 2

search-query-auto-complete: Striver A2Z: First question

In [18]:
class AutoCompleteSystem:
    def __init__(self, sentences: list[str], times: list[int]) -> None:
        # Store a dictionary of sentences along with their frequencies
        self.sentences: collections.defaultdict[str, int] = collections.defaultdict(lambda: 0)
        self.acc: str = ""
        for sentence, freq in zip(sentences, times):
            self.sentences[sentence] = freq

    def input(self, character: str) -> list[str]:
        if character != '#':
            self.acc += character
            result: list[tuple[int, str]] = []
            for sentence, freq in self.sentences.items():
                if sentence.startswith(self.acc):
                    result.append((-freq, sentence))
            result.sort()
            return list(map(lambda x: x[1], result[:3]))

        else:
            self.sentences[self.acc] += 1
            self.acc = ''
            return []

# Testing the solution - 1
temp = AutoCompleteSystem(["i love you", "island", "ironman", "i love geeksforgeeks"], [5,3,2,2])
assert temp.input("i") == ["i love you", "island","i love geeksforgeeks"]
assert temp.input(" ") == ["i love you", "i love geeksforgeeks"]
assert temp.input("a") == []
assert temp.input("#") == []
assert temp.input("i") == ['i love you', 'island', 'i love geeksforgeeks']
assert temp.input(" ") == ['i love you', 'i love geeksforgeeks', 'i a']
assert temp.input("a") == ['i a']
assert temp.input("m") == []
assert temp.input("#") == []
assert temp.input("i") == ['i love you', 'island', 'i love geeksforgeeks']
assert temp.input(" ") == ['i love you', 'i love geeksforgeeks', 'i a']
assert temp.input("a") == ['i a', 'i am']
assert temp.input("m") == ['i am']
assert temp.input(" ") == []

Patterns practice

In [19]:
def p1(N: int) -> None:
    for i in range(N):
        for j in range(N):
            print('*', end='')
        print()

# Testing the solution
p1(5)

*****
*****
*****
*****
*****


In [20]:
def p2(N: int) -> None:
    for i in range(N):
        for j in range(i + 1):
            print("*", end='')
        print()

# Testing the solution
p2(5)

*
**
***
****
*****


In [21]:
def p3(N: int) -> None:
    for i in range(N):
        for j in range(i + 1):
            print(j + 1, end=' ')
        print()

# Testing the solution
p3(5)

1 
1 2 
1 2 3 
1 2 3 4 
1 2 3 4 5 


In [22]:
def p4(N: int) -> None:
    for i in range(N):
        for j in range(i + 1):
            print(i + 1, end=' ')
        print()

# Testing the solution
p4(5)

1 
2 2 
3 3 3 
4 4 4 4 
5 5 5 5 5 


In [23]:
def p5(N: int) -> None:
    for i in range(N, 0, -1):
        for j in range(i):
            print("*", end='')
        print()

# Testing the solution
p5(5)

*****
****
***
**
*


In [24]:
def p6(N: int) -> None:
    for i in range(N, 0, -1):
        for j in range(i):
            print(j + 1, end='')
        print()

# Testing the solution
p6(5)

12345
1234
123
12
1


In [25]:
def p7(N: int) -> None:
    max_stars: int = (2 * N) - 1
    for i in range(1, N + 1):
        stars: int = (2 * i) - 1
        half = (max_stars - stars) // 2
        for j in range(half):
            print(" ", end='')
        for j in range(stars):
            print("*", end='')
        for j in range(half):
            print(" ", end='')
        print()

# Testing the solution
p7(5)

    *    
   ***   
  *****  
 ******* 
*********


In [26]:
def p8(N: int) -> None:
    max_stars: int = (2 * N) - 1
    for i in range(N, 0, -1):
        stars: int = (2 * i) - 1
        half = (max_stars - stars) // 2
        for j in range(half):
            print(" ", end='')
        for j in range(stars):
            print("*", end='')
        for j in range(half):
            print(" ", end='')
        print()

# Testing the solution
p8(5)

*********
 ******* 
  *****  
   ***   
    *    


In [27]:
def p9(N: int) -> None:
    p7(N)
    p8(N)

# Testing the solution
p9(5)

    *    
   ***   
  *****  
 ******* 
*********
*********
 ******* 
  *****  
   ***   
    *    


In [28]:
def p10(N: int) -> None:
    for i in range(N):
        for j in range(i + 1):
            print("*", end='')
        print()

    for i in range(N - 1, 0, -1):
        for j in range(i):
            print("*", end='')
        print()

# Testing the solution
p10(5)

*
**
***
****
*****
****
***
**
*


In [29]:
def p11(N: int) -> None:
    for i in range(N):
        for j in range(i + 1):
            print(1 if i % 2 == j % 2 else 0, end=" ")
        print()

# Testing the solution
p11(5)

1 
0 1 
1 0 1 
0 1 0 1 
1 0 1 0 1 


In [30]:
def p12(N: int) -> None:
    for i in range(1, N + 1):
        for j in range(1, N + 1):
            print(j if i >= j else " ", end="")
        for j in range(N, 0, -1):
            print(j if i >= j else " ", end="")
        print()

# Testing the solution
p12(5)

1        1
12      21
123    321
1234  4321
1234554321


In [31]:
def p13(N: int) -> None:
    counter = 1
    for i in range(1, N + 1):
        for j in range(i):
            print(counter, end=" ")
            counter += 1
        print()

# Testing the solution
p13(5)

1 
2 3 
4 5 6 
7 8 9 10 
11 12 13 14 15 


In [32]:
def p14(N: int) -> None:
    for i in range(1, N + 1):
        for j in range(i):
            print(chr(ord('A') + j), end=" ")
        print()

# Testing the solution
p14(5)

A 
A B 
A B C 
A B C D 
A B C D E 


In [33]:
def p15(N: int) -> None:
    for i in range(N, 0, -1):
        for j in range(i):
            print(chr(ord('A') + j), end=" ")
        print()

# Testing the solution
p15(5)

A B C D E 
A B C D 
A B C 
A B 
A 


In [34]:
def p16(N: int) -> None:
    for i in range(1, N + 1):
        for j in range(i):
            print(chr(ord('A') + i - 1), end=" ")
        print()

# Testing the solution
p16(5)

A 
B B 
C C C 
D D D D 
E E E E E 


In [35]:
def p17(N: int) -> None:
    max_width = (2 * N) - 1
    for i in range(1, N + 1):
        width = (2 * i) - 1
        spaces = max_width - width
        for j in range(spaces // 2):
            print(" ", end="")
        for j in range(i):
            print(chr(ord('A') + j), end="")
        for j in range(i - 2, -1, -1):
            print(chr(ord('A') + j), end="")
        for j in range(spaces // 2):
            print(" ", end="")
        print()

# Testing the solution
p17(5)

    A    
   ABA   
  ABCBA  
 ABCDCBA 
ABCDEDCBA


In [36]:
def p18(N: int) -> None:
    for i in range(1, N + 1):
        for j in range(N - i, N):
            print(chr(ord('A') + j), end=' ')
        print()

# Testing the solution
p18(5)

E 
D E 
C D E 
B C D E 
A B C D E 


In [37]:
def p19(N: int) -> None:
    max_width = 2 * N
    for i_ in range(2 * N):
        i = N - i_ if i_ < N else (i_ % N) + 1
        for j in range(i):
            print("*", end='')
        for j in range(max_width - (2 * i)):
            print(" ", end='')
        for j in range(i):
            print("*", end='')
        print()

# Testing the solution
p19(5)

**********
****  ****
***    ***
**      **
*        *
*        *
**      **
***    ***
****  ****
**********


In [38]:
def p20(N: int) -> None:
    max_width = 2 * N
    for i_ in range(2 * N - 1):
        i = i_ + 1 if i_ < N else 2 * N - i_ - 1
        for j in range(i):
            print("*", end='')
        for j in range(max_width - 2 * i):
            print(" ", end='')
        for j in range(i):
            print("*", end='')
        print()

# Testing the solution
p20(5)

*        *
**      **
***    ***
****  ****
**********
****  ****
***    ***
**      **
*        *


In [39]:
def p21(N: int) -> None:
    for i in range(N):
        for j in range(N):
            print("*" if i in (0, N - 1) or j in (0, N - 1) else " ", end='')
        print()

# Testing the solution
p21(5)

*****
*   *
*   *
*   *
*****


In [40]:
def p22(N: int) -> None:
    result: collections.deque[collections.deque[int]] = collections.deque([collections.deque([1])])
    for i in range(2, N + 1):
        for row in result:
            row.appendleft(i)
            row.append(i)
        result.appendleft(collections.deque([i for _ in range(2 * i - 1)]))
        result.append(collections.deque([i for _ in range(2 * i - 1)]))

    for row in result:
        for col in row:
            print(col, end=' ')
        print()

# Testing the solution
p22(3)

3 3 3 3 3 
3 2 2 2 3 
3 2 1 2 3 
3 2 2 2 3 
3 3 3 3 3 


Basic Math

In [41]:
def reverseDigits(N: int) -> int:
    NR, num = 0, abs(N)
    while num > 0:
        digit, num = num % 10, num // 10
        NR = (NR * 10) + digit
    return NR if N > 0 else -NR

# Testing the solution
assert reverseDigits(-123) == -321

In [42]:
def isPalindrome(N: int) -> bool:
    def reverse(num: int) -> int:
        numR = 0
        while num > 0:
            digit, num = num % 10, num // 10
            numR = (numR * 10) + digit
        return numR

    return N >= 0 and N == reverse(N)

# Testing the solution
assert isPalindrome(121) == True

def lcmAndGcd(n1: int , n2: int) -> tuple[int, int]:
    def primeFactors(n: int) -> dict[int, int]:
        factors: collections.defaultdict[int, int] = collections.defaultdict(int)
        curr = 2
        while curr <= n:
            if n % curr == 0:
                n //= curr
                factors[curr] += 1
            else:
                curr += 1
        return factors

    pf1, pf2 = primeFactors(n1), primeFactors(n2)
    factors: set[int] = {*pf1.keys(), *pf2.keys()}
    lcm = gcd = 1
    for factor in factors:
        min_, max_ = min(pf1[factor], pf2[factor]), max(pf1[factor], pf2[factor])
        if min_ != 0:
            gcd *= factor ** min_
        if max_ != 0:
            lcm *= factor ** max_

    return lcm, gcd

Testing the solution
assert lcmAndGcd(5, 10) == (10, 5)
assert lcmAndGcd(14, 8) == (56, 2)
assert lcmAndGcd(8, 42) == (168, 2)

In [43]:
def lcmAndGcdOptimal(n1: int , n2: int) -> tuple[int, int]:
    """
    Euclidean algo: GCD remains same even if greater of two numbers is subtracted with the smaller number
    Continously subtract greater by smaller until we get a 0.
    """
    def gcd(n1: int, n2: int) -> int:
        while n1 != 0 and n2 != 0:
            n1, n2 = sorted([n1, n2])
            n1, n2 = n1, n2 - n1
        return max(n1, n2)

    gcd_ = gcd(n1, n2)
    lcm_ = (n1 * n2) // gcd_
    return lcm_, gcd_

# Testing the solution
assert lcmAndGcdOptimal(5, 10) == (10, 5)
assert lcmAndGcdOptimal(14, 8) == (56, 2)
assert lcmAndGcdOptimal(8, 42) == (168, 2)

In [44]:
def armstrong(N: int) -> bool:
    def countDigits(n: int) -> int:
        digits = 0
        while n > 0:
            digits, n = digits + 1, n // 10
        return digits

    total, n, K = 0, N, countDigits(N)
    while n > 0:
        digit, n = n % 10, n // 10
        total +=digit ** K

    return total == N

# Testing the solution
assert armstrong(371) == True
assert armstrong(123) == False

In [45]:
def printAllDivisors(N: int) -> list[int]:
    result: set[int] = set()
    i = 1
    while i * i <= N:
        if N % i == 0:
            result.add(i)
            result.add(N // i)
        i += 1

    return sorted(result)

# Testing the solution
assert printAllDivisors(12) == [1, 2, 3, 4, 6, 12]

In [46]:
def minJumps(arr: list[int], N: int) -> int:
    """
    Greedy Approach = Time: O(N), Space: O(1)

    Keep track of the range we can jump into with minimum number of steps
    For jump = 0, we can jump to index = 0 (left = right = 0)

    For each iteration through range, compute maximum range we can jump into.
    As we move with the next iteration, jumps = jumps + 1.
    Again recompute the next range
    """
    jumps = left = right = 0
    while right < N - 1:
        max_idx = 0
        for i in range(left, right + 1):
            max_idx = max(max_idx, i + arr[i])
        # For next iteration, we start with where we left off in the previous range (right + 1)
        jumps, left, right = jumps + 1, right + 1, max_idx

    return jumps

# Testing the solution
assert minJumps([1,3,5,8,9,2,6,7,6,8,9], 11) == 3